In [71]:
import pandas as pd
import numpy as np

class AdaBoosting:
    def __init__(self, K):
        self.K = K
        self.w = []
        pass
    
    def train(self, X_train, Y_train):
        total = len(Y_train)
        D = np.ones(total)/total
        self.h = []
        for k in range(self.K):
            ds = DecisionStump()
            ds.train(D, X_train, Y_train)
            print('k=',k, ', feature=',ds.i, ds.b, ds.theta)
            self.h.append(ds)
            epsilon = ds.error_rate(D, X_train, Y_train)
            self.w.append(np.log(1/epsilon - 1)/2)
            D_total = 0
            for j in range(total):
                D_total = D_total + D[j] * np.exp(-self.w[k] * Y_train[j] * ds.output(X_train[j]))
            for j in range(total):
                D[j] = D[j] * np.exp(-self.w[k] * Y_train[j] * ds.output(X_train[j])) / D_total
        pass
    
        
    def output(self, X):
        h_b = 0
        for k in range(self.K):
            h_b = h_b + self.w[k]*self.h[k].output(X)
        return np.sign(h_b)
    
    def error_rate(self, X_test, Y_test):
        error = 0
        total = len(Y_test)
        for j in range(total):
            if self.output(X_test[j]) != Y_test[j]:
                error = error + 1
        return error/total


class DecisionStump:
    def __init__(self):
        pass
    
    def train(self, D, X_train, Y_train):
        data_train = np.concatenate((X_train, Y_train), axis=1)
        feature_len = len(X_train[0])
        best_rate = len(Y_train)
        for feature in range(feature_len):
            sort_data = data_train[data_train[:,feature].argsort()]
            theta_crt = sort_data[0][feature] - 1
            l0 = 0
            l1 = 1
            r0 = 0
            r1 = 0
            for m in range(len(Y_train)):
                if data_train[m][feature_len] == 0:
                    r0 = r0 + D[m]
                else:
                    r1 = r1 + D[m]
            if r0<r1:
                i = feature
                b = 1
                theta = theta_crt
                best_rate = r0
            else:
                i = feature
                b = -1
                theta = theta_crt
                best_rate = r1
            
            j = 0
            while j < len(sort_data):
                theta_crt = sort_data[j][feature]
                while j < len(sort_data) and theta_crt==sort_data[j][feature]:
                    if sort_data[j][feature_len] == 0:
                        l0 = l0 + D[j]
                        r0 = r0 - D[j]
                    else:
                        l1 = l1 + D[j]
                        r1 = r1 - D[j]
                    j = j + 1
                if best_rate > l1 + r0:
                    i = feature
                    b = 1
                    theta = theta_crt
                    best_rate = l1 + r0
                if best_rate > l0 + r1:
                    i = feature
                    b = -1
                    theta = theta_crt
                    best_rate = l0 + r1
            #print(best_rate, i, theta, b)
        
        self.i = i
        self.theta = theta
        self.b = b
        
    def output(self, X):
        if X[self.i]- self.theta>0:
            return max(self.b, 0)
        else:
            return max(0, -self.b)
    
    def error_rate(self, D, X_test, Y_test):
        error = 0
        total = len(Y_test)
        for j in range(total):
            if self.output(X_test[j]) != Y_test[j]:
                error = error + D[j]
        return error

In [72]:
ab = AdaBoosting(15)
ab.train(X_train, Y_train)
print(ab.error_rate(X_test, Y_test))

k= 0 , feature= 9 -1 0
k= 1 , feature= 9 -1 0
k= 2 , feature= 9 -1 0
k= 3 , feature= 9 -1 0
k= 4 , feature= 9 -1 0
k= 5 , feature= 9 -1 0
k= 6 , feature= 9 -1 0
k= 7 , feature= 9 -1 0
k= 8 , feature= 9 -1 0
k= 9 , feature= 9 -1 0
k= 10 , feature= 9 -1 0
k= 11 , feature= 9 -1 0


KeyboardInterrupt: 

In [60]:
ds = DecisionStump()
D = np.ones(len(Y_train))/len(Y_train)
print(D)
ds.train(D, X_train, Y_train)
print(len(Y_train[Y_train[:,0]==0]))
print(len(Y_train[Y_train[:,0]==1]))
print(ds.error_rate(np.ones(len(Y_test))/len(Y_test), X_test, Y_test))

[3.31543001e-05 3.31543001e-05 3.31543001e-05 ... 3.31543001e-05
 3.31543001e-05 3.31543001e-05]
0
0.2489224852463116 0 16 -1
1
0.2489224852463116 1 0 -1
2
0.2489224852463116 2 0 -1
3
0.2489224852463116 3 0 -1
4
0.2489224852463116 4 0 -1
5
0.2489224852463116 5 0 -1
6
0.2489224852463116 6 0 -1
7
0.2489224852463116 7 0 -1
8
0.2489224852463116 8 0 -1
9
0.2489224852463116 9 0 -1
22654
7508
0.24568393094291138


In [4]:
import pandas as pd
def read_data(s, delete_name):
    '''
    s, <string>, which dataset to access
    delete_name, <set>, which features do not take into account
    '''
    if s=='test':
        df = pd.read_csv('pro_data_test.csv', delimiter=',')
    elif s=='train':
        df = pd.read_csv('pro_data_train.csv', delimiter=',')
    else:
        return
    columns = df.columns
    for column in columns:
        if column in delete_name:
            del df[column]
    #print(df.head())
    feature_len = len(df.columns)-1
    feature = df[df.columns[0:feature_len-1]]
    label = df[df.columns[feature_len:feature_len+1]]
    X = feature.values
    Y = label.values
    return X, Y

In [5]:
delete_name = {'fnlwgt', 'capital-gain', 'capital-loss'}
X_train, Y_train = read_data('train', delete_name)
X_test, Y_test = read_data('test', delete_name)
#print(len(X_train[0]), len(X_test[0]))

In [11]:
print(X_test.shape)
print(Y_test.shape)
test = np.concatenate((X_test, Y_test), axis=1)
print(test.shape)
print(X_test[0:3])
print(Y_test[0:3])
print(test[0:3])
test[:,2].argsort()
#print(test[0:3])

(15060, 10)
(15060, 1)
(15060, 11)
[[25  1  3  7  3  8  2  5  2 40]
 [38  1  4  9  1 10  3  1  2 50]
 [28  5  6 12  1 13  3  1  2 40]]
[[0]
 [0]
 [1]]
[[25  1  3  7  3  8  2  5  2 40  0]
 [38  1  4  9  1 10  3  1  2 50  0]
 [28  5  6 12  1 13  3  1  2 40  1]]


array([ 7529,  6438,  6443, ..., 13100, 10607, 14354])

In [38]:
dataf = pd.read_csv('adult.csv', delimiter=', ')
print(dataf[dataf['<=50K'] == '<=50K'].shape)
print(dataf.shape)

/Users/ruida/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


(24719, 15)
(32560, 15)


[3.31543001e-05 3.31543001e-05 3.31543001e-05 ... 3.31543001e-05
 3.31543001e-05 3.31543001e-05]
0
0.2489224852463116 0 16 -1
1
0.2489224852463116 1 0 -1
2
0.2489224852463116 2 0 -1
3
0.2489224852463116 3 0 -1
4
0.2489224852463116 4 0 -1
5
0.2489224852463116 5 0 -1
6
0.2489224852463116 6 0 -1
7
0.2489224852463116 7 0 -1
8
0.2489224852463116 8 0 -1
9
0.2489224852463116 9 0 -1
22654
7508


NameError: name 'K' is not defined